In [1]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

from pandas import DataFrame, read_csv

In [ ]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

In [ ]:
#threshold

def test_valid_window(window):
    """
    window : the window in the signal that has to be tested
    
    This funtion test the window to insure that it doesn't contain the signal of interest (spike)
    """
    #non zero ?
    if np.percentile(window, 2)/np.percentile(window, 30) < 5 : return True
    else : return False
    

def init_noise_levels(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8):
    valid_windows = 0
    initial_thresholds = []
    noise_levels = []

    for window in range(0,len(signal),fs*noise_window_size):
        if valid_windows < required_valid_windows:
            test = test_valid_window(signal[window, window + fs*noise_window_size])
        
            if test == True :
                threshold = np.sqrt(np.mean(signal[window, window + fs*noise_window_size]**2))
                initial_threshold.append(threshold)
                valid_windows += 1
            
            if valid_windows == required_valid_windows:
                noise_level = np.percentile(initial_thresholds, 25)
                for elm in range(0, window):
                    noise_levels.append(noise_level)
                
        else :
            #ajouter le test valid_window ?
            N25 = np.percentile(signal[window, window + fs*noise_window_size], 25)
            noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
            noise_levels.append(noise_level)
        
    return noise_levels
            

In [1]:
#find spike

def find_spike(signal, initial, noise_levels, fs, spike_centers, 
               window_size = 0.001, 
               noise_window_size = 0.01,
               threshold_factor = 4):
    if initial < len(signal):
        i = initial
        for value in signal[initial:]:
            if value > threshold_factor*noise_levels[(i/noise_window_size)//fs]:
                while(True):
                    if i > initial + window_size*fs:
                        b_point = i - window_size*fs
                    else :
                        b_point = initial
                    if i < len(signal)-window_size*fs-1:
                        e_point = i + window_size*fs
                    else :
                        e_point = len(signal)-1

                    highest_value = max(signal[b_point, e_point])

                    if highest == value : # && 50% amplitude
                        spike_centers.append(i)
                        break
                    
                    else:
                        #i = signal.index.get_loc(highest)
                        i = signal[b_point, e_point].index(max(signal[b_point, e_point]))+b_point
                        value = signal[i]
                        
                break
                find_spike(signal, e_point+1, noise_levels, fs, spike_centers)
                    
            if value < -threshold_factor*noise_levels[(i/noise_window_size)//fs]:
                while(True):
                    if i > initial + window_size*fs:
                        b_point = i - window_size*fs
                    else :
                        b_point = initial
                    if i < len(signal)-window_size*fs-1:
                        e_point = i + window_size*fs
                    else :
                        e_point = len(signal)-1

                    lowest_value = min(signal[b_point, e_point])

                    if lowest == value : # && 50% amplitude
                        spike_centers.append(i)
                        break
                    
                    else:
                        #i = signal.index.get_loc(lowest)
                        i = signal[b_point, e_point].index(min(signal[b_point, e_point]))+b_point
                        value = signal[i]
                        
                break
                find_spike(signal, e_point+1, noise_levels, fs, spike_centers)
                  
            i += 1

0
1
2
3
4
5
6
7
8
9


In [ ]:
#record spike

def record_spikes(signal, fs, spike_centers, 
                  t_before = 0.001, 
                  t_after = 0.002):
    data = np.array([x for x in range(fs*(t_before+t_after)+1)])
    
    for center in spike_centers:
        if center < fs*t_before:
            spike = [0 for i in range(0, fs*t_before-center)]
            spike = np.concatenate(spike, signal[:center + fs*t_after])
            np.insert(data, len(data), spike, axis=0)
            
        else if center > len(signal)-fs*t_after:
            spike = signal[center-fs*t_before:]
            spike = np.concatenate(spike,[0 for i in range(0, fs*t_after-(len(signal)-center))])
            np.insert(data, len(data), spike, axis=0)
            
        else :
            spike = signal[center - fs*t_before: center + fs*t_after]
            np.insert(data, len(data), spike, axis=0)
            
    spike_data = pd.DataFrame(data[:,1:], index = data[:,0], columns=spike_centers)
    
    return spike_data

In [7]:
# test of the functions

signal = df.iloc[:,1]
fs = 25000

noise_levels = init_noise_levels(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8)

spike_centers = []

find_spike(signal, 0, noise_levels, fs, spike_centers, 
               window_size = 0.001, 
               noise_window_size = 0.01,
               threshold_factor = 4)

spike_data = record_spikes(signal, fs, spike_centers, 
                  t_before = 0.001, 
                  t_after = 0.002)

6

In [ ]:
plt.figure(figsize = (100,20))
plt.plot(spike_data.index, spike_data.iloc[:,1])
plt.xlabel('Time')
plt.grid()